In [7]:
import math
import torch 
import torch.nn as nn

import torch.nn.functional as F

In [2]:
class BasicBlock(nn.Module):

    def __init__(self, in_f , out_f , dropRate = 0.0):

        super(BasicBlock , self).__init__()

        self.bn1 = nn.BatchNorm2d(in_f)
        # inplace ==> if false it creates a new tensor and performs batch nrom on that if True it performs batch nomr the given tensor
        self.a1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels=in_f , out_channels=out_f , kernel_size=3 , stride=1 , padding=1 , bias=False)
        self.droprate = dropRate


    def forward(self , x):

        out = self.conv1(self.a1(self.bn1(x)))

        if self.droprate > 0:
            out = F.dropout(out , p=self.droprate , training=self.training)

        return torch.cat([x , out] , 1)
    


In [4]:
class BottleNeck(nn.Module):

    def __init__(self , in_f , out_f , dropRate = 0.0):
        super(BottleNeck , self).__init__()
        inter_f = out_f * 4
        self.bn1 = nn.BatchNorm2d(in_f)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_f, inter_f, kernel_size=1, stride=1,
                               padding=0, bias=False)
        

        self.bn2 = nn.BatchNorm2d(inter_f)
        self.conv2 = nn.Conv2d(inter_f, out_f, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate

    def forward(self , x):

        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)

        out = self.conv2(self.relu(self.bn2(out)))

        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)

        return torch.cat([x , out] , 1)
    




        

        

In [5]:
class TransitionBlock(nn.Module):

    def __init__(self , in_f , out_f , dropRate=0.0):

        super(TransitionBlock  , self).__init__()

        self.bn1 = nn.BatchNorm2d(in_f)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_f , out_f , kernel_size=1 , stride=1 , padding=0 , bias=False)
        self.droprate = dropRate


    def forward(self , x):

        out = self.conv1(self.relu(self.bn1(x)))

        if self.droprate > 0:

            out = F.dropout(out , p=self.droprate , inplace=False , training=self.training)

        return F.avg_pool2d(out , 2)
    


In [6]:
class DenseBlock(nn.Module):

    def __init__(self, nb_layers , in_f , growth_rate , block , dropRate = 0.0):
        super(DenseBlock , self).__init__()
        self.layer = self._make_layer(block , in_f , growth_rate , nb_layers , dropRate)

    def _make_layer(self , block , in_f , growth_rate , nb_layers , dropRate):
        layers = []

        for i in range(nb_layers):
            layers.append(block(in_f + i*growth_rate , growth_rate , dropRate))

        return nn.Sequential(*layers)
    
    def forward(self , x):
        return self.layer(x)
    

    
                          

In [8]:
class DenseNet(nn.Module):

    def __init__(self , depth , num_classes , growth_rate = 12 , reduction = 0.5 , bottleneck = True , dropRate = 0.0):

        super(DenseNet , self).__init__()

        in_f = 2 * growth_rate
        n = (depth - 4)/3

        if bottleneck == True:
            n = n/2
            block = BottleNeck

        else:
            block = BasicBlock

        n = int(n)
        
        self.conv1 = nn.Conv2d( n ,in_f , growth_rate , block , dropRate)
        
        # 1st block
        self.block1 = DenseBlock(n , in_f , growth_rate , block , dropRate)
        in_f = int(in_f + n * growth_rate)
        self.trans1 = TransitionBlock(in_f , int(math.floor(in_f * reduction)) , dropRate=dropRate)
        in_f = int(math.floor(in_f*reduction))

        # 2nd block
        self.block2 = DenseBlock(n , in_f , growth_rate , block , dropRate)
        in_f = int(in_f + n*growth_rate)
        self.trans2 = TransitionBlock(in_f , int(math.floor(in_f *reduction )) , dropRate=dropRate)
        in_f = int(math.floor(in_f * reduction))

        # 3rd block

        self.block3 = DenseBlock(n , in_f , growth_rate , block , dropRate)
        in_f = int(in_f + n*growth_rate)

        # gloabal average pooling
        self.bn1 = nn.BatchNorm2d(in_f)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(in_f , num_classes)
        self.in_f = in_f

        # xavier initialization
        for m in self.modules():

            if isinstance(m , nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0 , math.sqrt(2. / n))

            elif isinstance( m  , nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

            elif isinstance(m , nn.Linear):
                m.bias.data.zero_()


    def forward(self ,x):
        out = self.conv1(x)
        out = self.trans1(self.block1(out))
        out = self.trans2(self.block2(out))
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.in_planes)
        return self.fc(out)
        